In [ ]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from numpy import array
import tensorflow as tf
import pickle
import keras
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import concatenate
from keras.layers.recurrent import LSTM,GRU
from sklearn.model_selection import TimeSeriesSplit
import time
import itertools
import os
import warnings
import io
import inspect
import random 
from random import randrange
import datetime
from datetime import datetime
from random import randrange
from tensorflow.keras.optimizers import Adam
from time import sleep
from tqdm import tqdm
from gurobipy import *
warnings.filterwarnings('ignore')
SIGNAL = pd.read_csv("20190806_bis_20210222_1s.csv")
forecast_name = {
    7: '7_0.1_30_4',
    8: '8_0.01_10_4',
    9: '9_0.1_30_12',
    10: '10_0.1_10_4',
    24: '24_0.1_10_4',
}
p_length = 7+1
t_length = 363*96+1

In [ ]:
def pre_opt(Us, U_s, SoC_s0 , SoC_min_s, SoC_max_s, P_max_s, E_max_s, mü, cost_work, cost_peak, capacity_payment, signal):
    
    """
    Optimize the reserved BESS capacities of E_freq_bw, C_bid_bw and C_bw
    ----------
    Parameters:
    Us : load of the day
    U_s: current shave level/ peak load
    SoC_s0: current SoC of the BESS, given by real-time control
    SoC_min_s: lower SoC limit
    SoC_max_s: upper SoC limit
    P_max_s: maximum power capacity of the BESS
    E_max_s: maximum energy capacity of the BESS (SoC limits already subtracted)
    mü: charge, discharge efficiency
    cost_work: electricity + working cost
    cost_peak: peak cost
    capacity_payment: FCR market price
    signal: expected maximum signal (90th-quantile of 2 year signal data)
    ----------
    return:  
    E_freq : 96-entry array with all the blocked energy capacities for the next day
    C : 96-entry array with all the blocked power capacities for the next day
    C_bid : 96-entry array with all capacity bids for the next day
    token : binary variable that indicates if optimization has valid solution
    """
    
    # Optimization (6) in the paper
    load_fc = Us.tolist()
    t_max = len(load_fc)

    s = Model("Prediction")
    s.setParam("OutputFlag",0)

    l = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, name = "l")
    discharge = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, name = "discharge")
    charge = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, name = "charge")
    SoC_s = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "SoC")
    max_l = s.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "max_l")
    E_freq_t= s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "E_freq_t")
    E_freq_bw = s.addVars(range(6), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "E_freq_bw")
    C_t = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "C_t")
    C_bw = s.addVars(range(6), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "C_bw")
    C_bid_t = s.addVars(range(t_max), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "C_bid_t")
    C_bid_bw = s.addVars(range(6), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "C_bid_bw")
    pos_diff = s.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "pos_diff")
    diff = s.addVar(vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY , name = "diff")

    s.setObjective((quicksum(l[t]/4*cost_work  for t in range(t_max)) + pos_diff*cost_peak - quicksum(C_bid_t[t]/4*capacity_payment for t in range(t_max))), GRB.MINIMIZE)

    s.addConstrs(( l[t] == ( load_fc[t]  + charge[t] - discharge[t] )  for t in range(t_max) ),"Last nach BESS-Einsatz")
    s.addConstrs(( l[t] >= 0  for t in range(t_max) ),"l_lb")
    
    s.addConstr(( max_l == max_(l[t] for t in range(t_max))),"ps_level einhalten")
    s.addConstr(diff == max_l-U_s)
    s.addConstr(pos_diff == max_(diff, constant=0))
    
    s.addConstrs(( charge[t] >= 0  for t in range(t_max) ),"charge lb")
    s.addConstrs(( charge[t] <= P_max_s-C_t[t]  for t in range(t_max) ),"charge ub")

    
    s.addConstrs(( discharge[t] >= 0  for t in range(t_max) ),"discharge lb")
    s.addConstrs(( discharge[t] <= P_max_s-C_t[t]  for t in range(t_max) ),"discharge ub1"))
    
    s.addConstrs(( SoC_s[t] >= SoC_min_s+E_freq_t[t]  for t in range(t_max) ),"SoC lb")
    s.addConstrs(( SoC_s[t] <= SoC_max_s-E_freq_t[t]  for t in range(t_max) ),"SoC ub")
    s.addConstr(( SoC_s[0] == SoC_s0  ),"SoC init")
    s.addConstrs(( SoC_s[t+1] == SoC_s[t] + (charge[t]*mü - discharge[t]/mü)/4  for t in range(t_max-1) ),"SoC update")
    s.addConstr(( SoC_s[t_max-1] >= E_max_s/2),"SoC out") 
    s.addConstr(( discharge[t_max-1] == 0  ),"SoC out")

    s.addConstrs(( E_freq_bw[bw] >= C_bw[bw]/4 for bw in range(0,6)),"E_freq_lb")
    s.addConstrs(( E_freq_bw[bw] <= E_max_s/2 for bw in range(0,6)),"E_freq_ub")
    
    s.addConstrs(( C_bw[bw] >= 0 for bw in range(0,6)),"C_lb")
    s.addConstrs(( C_bw[bw] <= P_max_s for bw in range(0,6)),"C_ub")
    
    s.addConstrs(( C_bid_bw[bw] >= 0 for bw in range(0,6)),"C_bid_lb")
    s.addConstrs(( C_bid_bw[bw] <= C_bw[bw]/signal for bw in range(0,6)),"C_bid_ub1")
    s.addConstrs(( C_bid_bw[bw] <= P_max_s for bw in range(0,6)),"C_bid_ub2")
    

    s.addConstrs(( E_freq_t[t] == E_freq_bw[0] for t in range(0,16) ),"E_freq1")
    s.addConstrs(( E_freq_t[t] == E_freq_bw[1] for t in range(16,32) ),"E_freq2")
    s.addConstrs(( E_freq_t[t] == E_freq_bw[2] for t in range(32,48) ),"E_freq3")
    s.addConstrs(( E_freq_t[t] == E_freq_bw[3] for t in range(48,64) ),"E_freq4")
    s.addConstrs(( E_freq_t[t] == E_freq_bw[4] for t in range(64,80) ),"E_freq5")
    s.addConstrs(( E_freq_t[t] == E_freq_bw[5] for t in range(80,96) ),"E_freq6")  

    s.addConstrs(( C_t[t] == C_bw[0] for t in range(0,16) ),"C1")
    s.addConstrs(( C_t[t] == C_bw[1] for t in range(16,32) ),"C2")
    s.addConstrs(( C_t[t] == C_bw[2] for t in range(32,48) ),"C3")
    s.addConstrs(( C_t[t] == C_bw[3] for t in range(48,64) ),"C4")
    s.addConstrs(( C_t[t] == C_bw[4] for t in range(64,80) ),"C5")
    s.addConstrs(( C_t[t] == C_bw[5] for t in range(80,96) ),"C6")

    s.addConstrs(( C_bid_t[t] == C_bid_bw[0] for t in range(0,16) ),"C_bid1")
    s.addConstrs(( C_bid_t[t] == C_bid_bw[1] for t in range(16,32) ),"C_bid2")
    s.addConstrs(( C_bid_t[t] == C_bid_bw[2] for t in range(32,48) ),"C_bid3")
    s.addConstrs(( C_bid_t[t] == C_bid_bw[3] for t in range(48,64) ),"C_bid4")
    s.addConstrs(( C_bid_t[t] == C_bid_bw[4] for t in range(64,80) ),"C_bid5")
    s.addConstrs(( C_bid_t[t] == C_bid_bw[5] for t in range(80,96) ),"C_bid6")

    s.optimize()
    
    E_freq = np.zeros(96)
    C = np.zeros(96)
    C_bid = np.zeros(96)
    SoC_so = np.zeros(96)
    discharge_o = np.zeros(96)
    charge_o = np.zeros(96)
    l_o = np.zeros(96)
    batt = np.zeros(96)
    E_freq_bwo = np.zeros(6)
    
    
    if s.solCount == 0:
        token = 1
        s.computeIIS()
        s.write("model_iis.ilp")
        for qh in range(96):
            E_freq[qh] = 0
            C[qh] = 0
            C_bid[qh] =0  
    else:
        token = 0
        for qh in range(96):
            E_freq[qh] = E_freq_t[qh].x
            C[qh] = C_t[qh].x
            C_bid[qh] = C_bid_t[qh].x
            SoC_so[qh] = SoC_s[qh].x
            discharge_o[qh] = discharge[qh].x
            charge_o[qh] = charge[qh].x
            l_o[qh] = l[qh].x
            batt[qh] = discharge_o[qh] - charge_o[qh]
    return E_freq, C, C_bid, token

        

def get_timewindow(df, start_year, start_month, start_day, start_hour, start_minute, stop_year, stop_month, stop_day, stop_hour, stop_minute):
    
    """
    Get the desired time window, to make sure the simulation start at 08:00 of the desired year
    ----------
    Parameters:
    df : dataframe of load data
    start_...: point of time to start slice
    stop_...: point of time to stop slice
    ----------
    return:  
    df_window: df of desired time window
    """
    
    start_row = 0
    for i in df.index:
        start_row = start_row+1
        if i.year == start_year and i.month == start_month and i.day == start_day and i.hour == start_hour and i.minute == start_minute:
            break
    start_row = start_row-1
    start_index = i
    stop_row = 0
    for j in df.index:
        stop_row = stop_row+1
        if j.year == stop_year and j.month == stop_month and j.day == stop_day and j.hour == stop_hour and j.minute == stop_minute:
            break
    stop_index = j
    df_window = df.iloc[start_row:stop_row,:]
    return df_window


def ann(f, l, i):
    """
    Compute annuity
    ----------
    Parameters:
    f: fix cost of investment (I0)
    l: lifetime of investment (BESS lifespan)
    i: intesrest rate
    ----------
    return:  
    annuity factor
    """
    return f * (i*((1+i)**l) / ((1+i)**l-1))

In [ ]:
# SIMULATION


for p_fcr in [0.012]: # iterate with different FCR market prices e.g. [0.008, 0.01, 0.012, 0.014, 0.016, 0.019]
    for p_batt in [120]: # iterate with different BESS prices e.g. [150, 120, 100]
        result_latex = {}
        for c in range(5):
            result_latex[c] = pd.DataFrame()
            
        for size_mod in [1]: # iterate with different BESS size modifications e.g.[0.9, 1, 1.1]
            
            level_mod = 1 # choose different shave level modifications e.g. 0.5, 0.75, 1

            
            all_load = {}
            year_before = {}
            
            # get load data of the year before simulation
            for enter_company in (7,8,9,10,24):
                year_before[enter_company] = pd.DataFrame()
                year_before[enter_company] = pd.read_pickle(fr"__path__\v2_UN{enter_company}.pkl")
                year_before[enter_company] = get_timewindow(year_before[enter_company], 2015, 1, 2, 8, 0, 2015, 12, 31, 8, 0)
            # get load data and forecasts of the year to be simulated
                all_load[enter_company] = pd.DataFrame()
                all_load[enter_company] = pd.read_pickle(fr"__path__\LSTM{forecast_name[enter_company]}.pkl")
                oe = all_load[enter_company]["95%"].max()*2
                ue = 0.001
                overestimate = np.zeros(all_load[enter_company].shape[0])
                underestimate = np.zeros(all_load[enter_company].shape[0])
                for e in range(all_load[enter_company].shape[0]):
                    overestimate[e] = oe
                    underestimate[e] = ue
                all_load[enter_company]['ps_only'] = overestimate
                all_load[enter_company]['fcr_only'] = underestimate
                all_load[enter_company] = get_timewindow(all_load[enter_company], 2016, 1, 2, 8, 0, 2016, 12, 31, 8, 0)

            U = np.zeros(shape=(5,p_length,all_load[enter_company].shape[0]))
            U_yb = np.zeros(shape=(5, year_before[enter_company].shape[0]))
            U_yb[0] =  year_before[7].iloc[:,0]
            U_yb[1] =  year_before[8].iloc[:,0]
            U_yb[2] =  year_before[9].iloc[:,0]
            U_yb[3] =  year_before[10].iloc[:,0]
            U_yb[4] =  year_before[24].iloc[:,0]

            for a in range(p_length):
                U[0][a] = all_load[7].iloc[:,a]
                U[1][a] = all_load[8].iloc[:,a]
                U[2][a] = all_load[9].iloc[:,a]
                U[3][a] = all_load[10].iloc[:,a]
                U[4][a] = all_load[24].iloc[:,a]
                
            
            # bring negative forecast values to 0
            for c in range(5):
                U_yb[c] = [0 if i < 0 else i for i in U_yb[c]] 
                for p in range(8):
                    U[c][p] = [0 if i < 0 else i for i in U[c][p]]

            # Variable declaration
            SoC_min = np.zeros(5)
            SoC_max = np.zeros(5)
            E_max = np.zeros(5)
            P_max = np.zeros(5)
            opt_capacity = np.zeros(5)
            opt_power = np.zeros(5)
            opt_level = np.zeros(5)
            S = np.zeros(shape=(5,p_length,t_length))
            U_ = np.zeros(shape=(5,p_length,t_length))
            b = np.zeros(shape=(5,p_length,t_length))
            b_ch = np.zeros(shape=(5,p_length,t_length))
            b_dc = np.zeros(shape=(5,p_length,t_length))
            SoC = np.zeros(shape=(5,p_length,t_length))
            E_freq = np.zeros(shape=(5,p_length,t_length))
            C = np.zeros(shape=(5,p_length,t_length))
            C_bid = np.zeros(shape=(5,10,t_length))
            E_freq_d = np.zeros(6)
            C_d = np.zeros(6)
            C_bid_d = np.zeros(6)
            SoC_space_d = np.zeros(6)
            c_elec = np.zeros(shape=(5,p_length))
            c_peak = np.zeros(shape=(5,p_length))
            r_fcr = np.zeros(shape=(5,p_length))
            c_total = np.zeros(shape=(5,p_length)) 
            bw = np.array([0,16,32,48,64,80,96])   


            # constants
            life = 6 #11 years from Brauer et al. reduced by a factor found by luntz et al.
            interest = 0.02 #Brauer et al.
            p_pow = ann(p_batt*8/3,life,interest) #€/kW per year    simpkins (2017: 800$/kW, 300$/kWh)
            p_cap = ann(p_batt,life,interest) #€/kWh per year
            p_elec = 0.045 + 0.0056 #€/kWh
            p_peak = 86
            mü = 0.95
            signal = SIGNAL.quantile(0.9)[1]
            
            #ENTER COMPANY ITERATION
            for c in range(5):
                
                #BESS sizing is done on the previous year data U_yb
                load_m = U_yb[c].tolist()    
                #load_m = U[c][0].tolist()
                t_max_m = len(load_m)

                m = Model("BESS_sizing")
                m.setParam("OutputFlag",0)
                
                
                # Optimization (3) in the paper
                l_m = m.addVars(range(t_max_m), vtype = GRB.CONTINUOUS, lb = 0, ub = max(load_m), name = "l")
                power_m = m.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "power")
                capacity_m = m.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "capacity")
                discharge_m = m.addVars(range(t_max_m), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, name = "discharge")
                charge_m = m.addVars(range(t_max_m), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, name = "charge")
                SoC_m = m.addVars(range(t_max_m), vtype = GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY , name = "SoC")
                ps_level_m = m.addVar(vtype = GRB.CONTINUOUS, lb = 0, ub = max(load_m) , name = "ps_level")

                m.setObjective((quicksum(l_m[t]/4*p_elec  for t in range(t_max_m)) + ps_level_m*p_peak + p_pow*power_m + p_cap*capacity_m), GRB.MINIMIZE)

                m.addConstrs(( l_m[t] == ( load_m[t]  + charge_m[t] - discharge_m[t] )  for t in range(t_max_m) ),"Last nach BESS-Einsatz")
                m.addConstr(( ps_level_m == max_(l_m[t] for t in range(t_max_m))),"ps_level einhalten")
                m.addConstrs(( charge_m[t] >= 0  for t in range(t_max_m) ),"charge lb")
                m.addConstrs(( charge_m[t] <= power_m  for t in range(t_max_m) ),"charge ub")
                m.addConstrs(( discharge_m[t] >= 0  for t in range(t_max_m) ),"discharge lb")
                m.addConstrs(( discharge_m[t] <= power_m  for t in range(t_max_m) ),"discharge ub")
                m.addConstrs(( SoC_m[t] >= capacity_m*0.1  for t in range(t_max_m) ),"SoC lb")
                m.addConstrs(( SoC_m[t] <= capacity_m*0.95  for t in range(t_max_m) ),"SoC ub")
                m.addConstrs(( SoC_m[t+1] == SoC_m[t] + (charge_m[t]*mü - discharge_m[t]/mü)/4  for t in range(t_max_m-1) ),"SoC update")
                m.addConstrs(( SoC_m[t] == capacity_m*0.5 for t in range(0) ),"SoC init")

                m.optimize()
                
                # get capacity, power and shave level as output
                opt_capacity[c],opt_power[c],opt_level[c] = size_mod*capacity_m.X, size_mod*power_m.X, level_mod*ps_level_m.X


                #determine SoC limits, P_max and E_max
                SoC_min[c] = opt_capacity[c]*0.1
                SoC_max[c] = opt_capacity[c]*0.95
                E_max[c] = SoC_max[c] - SoC_min[c]
                P_max[c] = opt_power[c]
                U_help = np.full((1 , p_length, t_length), opt_level[c]) 
                U_[c] = U_help
                print("c:", c, opt_capacity[c], opt_power[c], opt_level[c])
                
                #ENTER PERCENTILE ITERATION
                for p in range(p_length):
                    token = 0
                    SoC[c][p][0] = E_max[c]/2
                    
                    #ENTER TIME PERIOD ITERATION BETWEEN DAYS
                    for n in range(0,t_length-1,96):
                        
                        # Day-ahead optimization is called and executed
                        if p <= p_length-3 : #for perfect foresight and percentiles
                            po = pre_opt(U[c][p][n:n+96], U_[c][p].max(), SoC[c][p][n] , SoC_min[c], SoC_max[c], P_max[c], E_max[c], mü, p_elec, p_peak, p_fcr, signal)
                            token = po[3]
                        if p == p_length-2: #for "peak shaving only"
                            po = np.zeros(shape=(3,96))
                            token = 0
                        if p == p_length-1: #for "FCR only"
                            po = np.zeros(shape=(3,96))
                            token = 0
                            for i in range(96):
                                po[2][i] = P_max[c]
                                po[1][i] = P_max[c]
                                po[0][i] = E_max[c]/2     
                                
                        # Day-ahead optimization results are handed over for the next 96 periods (1 day)
                        E_freq[c][p][n:n+96] = po[0]  
                        C[c][p][n:n+96] = po[1]
                        C_bid[c][p][n:n+96] = po[2]
                        
                        #ENTER TIME PERIOD ITERATION WITHIN A DAY
                        for qh in range(96):
                            t = n + qh
                            
                            # Real time control, described as Algorithm 1 in the paper
                            b[c][p][t] = U[c][0][t] - U_[c][p][t]
                            if b[c][p][t] >= 0:
                                b[c][p][t] = min(b[c][p][t], P_max[c]-C[c][p][t], max(0,(mü*(SoC[c][p][t]-(SoC_min[c]+E_freq[c][p][t]))/(1/4))), max(0,(mü*(SoC[c][p][t]-(SoC_min[c]+E_freq[c][p][t+1]))/(1/4))) )
                                b_dc[c][p][t] = b[c][p][t]
                                b_ch[c][p][t] = 0
                            else:
                                b[c][p][t] = max(b[c][p][t], -(P_max[c]-C[c][p][t]), min(0,((SoC[c][p][t]-(SoC_max[c]-E_freq[c][p][t]))/(mü*1/4))), min(0,((SoC[c][p][t]-(SoC_max[c]-E_freq[c][p][t+1]))/(mü*1/4))) )
                                b_dc[c][p][t] = 0
                                b_ch[c][p][t] = abs(b[c][p][t])

                            if qh >= 95: 
                                if SoC[c][p][t] > E_max[c]/2:
                                    b[c][p][t] = min(P_max[c]-C[c][p][t], mü*(SoC[c][p][t]-E_max[c]/2)/(1/4))
                                    b_dc[c][p][t] = b[c][p][t]
                                    b_ch[c][p][t] = 0
                                else:
                                    b[c][p][t] = max( -(P_max[c]-C[c][p][t]), (SoC[c][p][t]-E_max[c]/2)/(mü/4) )
                                    b_dc[c][p][t] = 0
                                    b_ch[c][p][t] = abs(b[c][p][t])


                            SoC[c][p][t+1] = SoC[c][p][t] + ((b_ch[c][p][t]*mü - b_dc[c][p][t]/mü)*(1/4))
                            S[c][p][t] = U[c][0][t] - b[c][p][t]
                            if S[c][p][t] > U_[c][p].max():
                                U_[c][p][t+1] = S[c][p][t]
                            else:
                                U_[c][p][t+1] = U_[c][p].max()

                    # Get notification if day-ahead optimization found no feasible solution
                    if token > 0:
                        print("                 p:", p ,"Optimierungsprobleme")
                    else:
                        print("                 p:", p)

                    c_elec[c][p] = S[c][p].sum()/4 * p_elec
                    c_peak[c][p] = U_[c][p].max() * p_peak
                    r_fcr[c][p] = C_bid[c][p].sum()/4 * p_fcr 
                    c_total[c][p] = c_peak[c][p] + c_elec[c][p] - r_fcr[c][p]


                print(size_mod, c_total[c][0:p_length])  

            compare = np.zeros(shape=(5,p_length))
            compare_ps_only = np.zeros(shape=(5,p_length))
            compare_fcr_only = np.zeros(shape=(5,p_length))
            no_batt = np.zeros(5)
            c_invest = np.zeros(5)
            c_invest_a2 = np.zeros(5)
            for c in range(5):
                no_batt[c] = U[c][0].max() * p_peak + U[c][0].sum() * p_elec/4
                c_invest[c] = p_batt*8/3*opt_power[c]+p_batt*opt_capacity[c]
                c_invest_a2[c] = ann(c_invest[c],life,interest)

            compare_no_batt = np.zeros(shape=(5,p_length))
            cashflow = np.zeros(shape=(5,p_length))
            NPV = np.zeros(shape=(5,p_length))
            PI = np.zeros(shape=(5,p_length))
            valid = np.zeros(5)

            def comp(a,b):
                return (a-b)/a
            
            # crf to calculate the NPV
            def crf(l, i):
                return (((1+i)**l-1) / (i*(1+i)**l))

            for x in range(5):
                for y in range(p_length):
                    compare[x][y] = comp(c_total[x][0],c_total[x][y])
                    
                    
        
        
        
            
            # COLLECT RESULTS AND PRINT TO EXCEL-TABLE
            for x in range(5):
                print([c_total[x][0] - c_total[x][1] , c_total[x][0] - c_total[x][2] , c_total[x][0] - c_total[x][3] , c_total[x][0] - c_total[x][4] , c_total[x][0] - c_total[x][5]])
                for y in range(p_length):
                    compare_ps_only[x][y] = comp(c_total[x][6],c_total[x][y])
                    compare_fcr_only[x][y] = comp(c_total[x][7],c_total[x][y])
                    compare_no_batt[x][y] = comp(no_batt[x],c_total[x][y])
                    cashflow[x][y] = no_batt[x] - c_total[x][y]
                    NPV[x][y] = -c_invest[x] + cashflow[x][y] * crf(life, interest)
                    PI[x][y] = NPV[x][y]/c_invest[x]

            #RESULT DFs
            no_batt_df = {}
            for c in range(5):
                no_batt_df[c] = pd.DataFrame()
                no_batt_df[c]["var cost"] = [no_batt[c]]
                no_batt_df[c]["peak cost"] =  U[c][0].max() * p_peak
                no_batt_df[c]["elec cost"] =  U[c][0].sum() * p_elec/4
                no_batt_df[c]["fcr revenue"] = 0
                no_batt_df[c]["new peak"] = U[c][0].max()
                no_batt_df[c]["comp perfect fs"] = ""
                no_batt_df[c]["comp only ps"] = ""
                no_batt_df[c]["comp only "] = ""
                no_batt_df[c]["total cost"] = no_batt[c] 
                no_batt_df[c]["NPV"] = 0
                no_batt_df[c]["PI"] = 0

            result_df = {}
            best_percentiles = np.zeros(5)
            for c in range(5):
                a = np.zeros(p_length)
                for p in range(p_length):
                    a[p] = U_[c][p].max()
                    
                result_df[c] = pd.DataFrame()
                result_df[c]["var cost"] = c_total[c][0:p_length]
                result_df[c]["peak cost"] = c_peak[c][0:p_length]
                result_df[c]["elec cost"] = c_elec[c][0:p_length]
                result_df[c]["fcr revenue"] = r_fcr[c][0:p_length]
                result_df[c]["new peak"] = a[0:p_length]
                result_df[c]["comp perfect fs"] = compare[c][0:p_length]
                result_df[c]["comp only ps"] = compare_ps_only[c][0:p_length]
                result_df[c]["comp only "] = compare_fcr_only[c][0:p_length] 
                result_df[c]["total cost"] = c_total[c][0:p_length]+[c_invest_a2[c] , c_invest_a2[c] , c_invest_a2[c] , c_invest_a2[c], c_invest_a2[c], c_invest_a2[c], c_invest_a2[c], c_invest_a2[c]]
                result_df[c]["NPV"] = NPV[c]
                result_df[c]["PI"] = PI[c]
                result_df[c] = pd.concat([result_df[c], no_batt_df[c]])
                result_df[c].index = ['perfect fs', '25th', '50th', '75th', '90th', '95th', 'ps only', 'fcr only', 'no batt']
                
                if size_mod == 1:
                    result_latex[c][f'PI {round(opt_capacity[c],2), round(opt_power[c],2)}'] = result_df[c]['PI']
                    result_latex[c]['total cost'] = result_df[c]['total cost']
                    result_latex[c]['peak cost'] = result_df[c]['peak cost']
                    result_latex[c]['fcr revenue'] = result_df[c]['fcr revenue']
                    result_latex[c]['elec cost'] = result_df[c]['elec cost']
                else:
                    result_latex[c][f'PI (size x{size_mod})'] = result_df[c]['PI']

        for c in range(5):
            result_latex[c].to_excel(fr'__path__\Comp{c}_fcr{p_fcr}_batt{p_batt}_lvl{level_mod}.xlsx')